In [1]:
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import sqlite3

In [2]:
response = requests.get("https://data.nba.net/10s/prod/v3/today.json")
today_json = response.json()
nba_data_links = today_json['links']

In [3]:
link_values = list(nba_data_links.values())
for link in link_values[2:-2]:
    print("https://data.nba.net{}".format(link))

https://data.nba.net/prod/v1/multi/calendar.json
https://data.nba.net/prod/v2/20210331/scoreboard.json
https://data.nba.net/prod/v2/20210331/scoreboard.json
https://data.nba.net/prod/v2/2020/teams.json
https://data.nba.net/prod/v2/{{gameDate}}/scoreboard.json
https://data.nba.net/prod/v1/2020/players.json
https://data.nba.net/prod/v1/allstar/2018/AS_roster.json
https://data.nba.net/prod/v1/2020/coaches.json
https://data.nba.net/prod/v1/multi/schedule.json
https://data.nba.net/prod/v1/current/standings_conference.json
https://data.nba.net/prod/v1/current/standings_division.json
https://data.nba.net/prod/v1/current/standings_all.json
https://data.nba.net/prod/v1/current/standings_all_no_sort_keys.json
https://data.nba.net/prod/v1/2020/team_stats_rankings.json
https://data.nba.net/prod/v1/2020/team_stats_last_five_games.json
https://data.nba.net/prod/v1/{{gameDate}}/{{gameId}}_preview_article.json
https://data.nba.net/prod/v1/{{gameDate}}/{{gameId}}_recap_article.json
https://data.nba.net

In [4]:
class DataNBANet:
    def __init__(self, year):
        self._year = year
    def get_teams(self):
        request_url = "https://data.nba.net/prod/v2/{}/teams.json".format(self._year)
        response = requests.get(request_url)
        teams_json = response.json()
        teams = [t for t in teams_json['league']['standard'] if t['isNBAFranchise']]
        teams_df = pd.DataFrame(teams)
        return teams_df
    def get_players(self):
        teams = self.get_teams()
        team_ids = teams['teamId'].values
        request_url = "https://data.nba.net/prod/v1/{}/players.json".format(self._year)
        response = requests.get(request_url)
        players_json = response.json()
        players_list = players_json['league']['standard']
        players_list_dict = []
        for p in tqdm(players_list):
            if p['teamId'] in team_ids:
                player_dict = {}
                for k, v in p.items():
                    if isinstance(v, str):
                        player_dict[k] = v
                players_list_dict.append(player_dict)
        players_df = pd.DataFrame(players_list_dict)
        return players_df
    def get_career_summaries(self):
        players = self.get_players()
        person_ids = players['personId']
        career_summaries = []
        for pid in tqdm(person_ids):
            request_url = "https://data.nba.net/prod/v1/{}/players/{}_profile.json".format(self._year, pid)
            response = requests.get(request_url)
            profile_json = response.json()
            career_summary = profile_json['league']['standard']['stats']['careerSummary']
            career_summaries.append(career_summary)
        career_summaries_df = pd.DataFrame(career_summaries)
        career_summaries_df.insert(0, 'personId', person_ids)
        career_summaries_df = career_summaries_df.replace('', np.nan)
        return career_summaries_df

In [5]:
dnn = DataNBANet(2020)
teams = dnn.get_teams()
players = dnn.get_players()
career_summaries = dnn.get_career_summaries()

100%|██████████| 484/484 [07:05<00:00,  1.14it/s]


In [6]:
conn = sqlite3.connect('nba.db')
teams.to_sql('teams', conn, if_exists='replace', index=False)
players.to_sql('players', conn, if_exists='replace', index=False)
career_summaries.to_sql('career_summaries', conn, if_exists='replace', index=False)